In [2]:
from typing import Union, Literal, TypedDict, Callable, Iterable, get_origin, get_args, Any, cast
from inspect import signature, Parameter
from types import NoneType

type Number = int | float
type Value = None | bool | Number | str | list["Value"] | dict[str, "Value"]
type ValueType = Literal["null", "boolean", "number", "string", "array", "object"]

class _JsonSchemaEnumTemplate[T: ValueType, S: Value](TypedDict):
    type: T
    enum: list[S]

class AssortedJsonSchemaEnum(TypedDict):
    enum: list[Value]

class JsonSchemaType(TypedDict):
    type: ValueType

type JsonSchemaEnum = (
    _JsonSchemaEnumTemplate[Literal["string"], str] |
    _JsonSchemaEnumTemplate[Literal["number"], Number] |
    _JsonSchemaEnumTemplate[Literal["null"], None] |
    _JsonSchemaEnumTemplate[Literal["boolean"], bool] |
    _JsonSchemaEnumTemplate[Literal["array"], list[Value]] |
    _JsonSchemaEnumTemplate[Literal["object"], dict[str, Value]] |
    AssortedJsonSchemaEnum
)

type JsonSchema = JsonSchemaType | JsonSchemaEnum

type JsonSchemaParameterAnnotation = Callable[[*tuple[str, ...]], int]

def to_openai_tool(function: Callable[..., str]):
    def uphold(condition: bool, message: str | None = None) -> bool:
        if message != None:
            assert condition, message
        else:
            assert condition
        
        return condition

    def to_json_schema(parameter_type: Any) -> JsonSchema:
        SCHEMA_TYPE_MAPPING: dict[type | None, ValueType] = {
            str: "string",
            int: "number",
            float: "number",
            dict: "object",
            list: "array",
            bool: "boolean",
            None: "null",
            NoneType: "null"
        }

        schema_type = SCHEMA_TYPE_MAPPING.get(parameter_type, None)
        
        if schema_type != None:
            return {"type": schema_type}
        
        def traverse_union(union_type: Any) -> Iterable[Value | type]:
            origin = get_origin(union_type)
            
            if origin in {Union, Literal}:
                for argument in get_args(union_type):
                    yield from traverse_union(argument)
                return
            
            yield union_type

        def unzip(iterable: Iterable[tuple[Any, ...]]) -> list[tuple[Any]]:
            return cast(list[tuple[Any]], zip(*iterable))

        arguments: tuple[Value, ...]
        argument_types: tuple[type[Value], ...]

        arguments, argument_types = unzip(
            (
                (argument, argument_type) for 
                argument, argument_type in (
                    (argument, type(argument)) for 
                    argument in 
                    traverse_union(parameter_type)
                ) if
                uphold(not isinstance(argument_type, type))
            )
        )

        enum_type: JsonSchemaEnum = {"enum": list(arguments)}

        arguments_type_iter = iter(argument_types)
        first_argument_type = next(arguments_type_iter)

        if all((
            issubclass(first_argument_type, argument_type) or issubclass(argument_type, first_argument_type) for 
            argument_type in 
            argument_types
        )):
            enum_type.update(to_json_schema(first_argument_type))

        return enum_type

    parameters = {
        parameter_name: to_json_schema(parameter_type.annotation) for 
        parameter_name, parameter_type in 
        signature(function).parameters.items() if
        uphold(parameter_type.annotation != Parameter.empty)
    }

    description = function.__doc__
    assert description != None

    return {
        "type": "function",
        "function": {
            "name": function.__name__,
            "description": description.strip(),
            "parameters": {
                "type": "object",
                "properties": parameters
            }
        }
    }

def get_weather_at(location: str, unit: Literal["celsius", "fahrenheit"] = "celsius") -> str:
    """Technically gets weather at the location"""

    return f"Weather there is the weather at {location} {unit}"

: 

In [5]:
from json import dumps as to_json_str
from pprint import pprint

weather_openai_function = to_openai_tool(get_weather_at)

serialized = None

try:
    serialized = to_json_str(weather_openai_function)
except Exception as e:
    serialized = str(e)

pprint(
    f"{weather_openai_function}"
)
print()
print(serialized)

("{'type': 'function', 'function': {'name': 'get_weather_at', 'description': "
 "'Technically gets weather at the location', 'parameters': {'type': 'object', "
 "'properties': {'location': <class 'str'>, 'unit': typing.Literal['celsius', "
 "'fahrenheit'], 'return': <class 'str'>}}}}")

Object of type type is not JSON serializable
